# RAG技术发展脉络

| 演进阶段 | 核心理念 | 核心技术手段 | 解决的主要痛点 | 局限性 |
| :--- | :--- | :--- | :--- | :--- |
| **阶段 1：朴素 RAG (Naive RAG)** | **基础检索-生成**：线性流水线，即“检索 -> 增强 -> 生成”。 | 1. 简单的文档分块 (Chunking)<br>2. 向量化 (Embedding)<br>3. 向量相似度搜索 | 解决了 LLM 知识时效性差、私有数据不可见的问题。 | 检索精度低（容易找错）、召回率差、生成内容容易产生幻觉。 |
| **阶段 2：高级 RAG (Advanced RAG)** | **精细化治理**：在检索前后增加大量策略，提升检索质量。 | 1. **检索前**：查询改写 (Query Rewrite)、多查询生成<br>2. **检索中**：混合检索 (Hybrid Search)、父子块检索<br>3. **检索后**：**重排序 (Re-ranking)**、上下文压缩 | 解决了检索结果相关性差的问题，减少了噪声干扰。 | 系统复杂度大幅增加，且仍是固定路径，无法处理复杂任务。 |
| **阶段 3：模块化 RAG (Modular RAG)** | **组件化架构**：功能解耦，根据需求灵活排列组合各模块。 | 1. **独立模块**：Rewrite、Search、Rerank、Verify<br>2. **流程重构**：引入多维检索、子查询分解<br>3. **GraphRAG**：引入知识图谱增强语义关联 | 提供了极高的灵活性，支持更复杂的数据类型（如图数据）。 | 模块间的协调成本高，对系统的架构设计要求较高。 |
| **阶段 4：智能体 RAG (Agentic RAG)** | **自主决策与闭环**：LLM 作为大脑，自主控制检索时机和方式。 | 1. **反思机制**：自我纠错 (Self-RAG)、输出验证<br>2. **多步推理**：多跳检索 (Multi-hop)、任务规划<br>3. **工具调用**：Agent 自动决定何时检索、何时总结 | 解决了复杂、跨文档问题的处理能力，实现了推理闭环。 | 计算资源开销大（Token 消耗高），且受限于 Agent 的推理稳定性。 |

### 技术演进背后的核心逻辑转变



1. **从“静态”到“动态”**：早期的 RAG 路径是死板的，现在的 RAG 可以根据问题的难易程度自动选择是否检索。
2. **从“相关性”到“事实性”**：早期只看向量像不像，现在通过重排序和反思机制，确保检索回来的信息必须是真实、准确、对回答有直接贡献的。
3. **从“碎片化”到“全局化”**：通过 **GraphRAG** 等技术，模型不再只是看几个碎片小块，而是能理解文档集合的全局逻辑。

# Naive RAG 流程

*Load ➡️ Split ➡️ Embed ➡️ Store ➡️ Retrieve ➡️ Augment ➡️ Generate*



### 1. Load：

[LangChain Document Loaders](https://python.langchain.com/docs/integrations/document_loaders/)

### 2. Split:

不同的文档切分策略

多模态的识别、切分 更复杂：OCR识别图中文字、CLIP、

### 3. Embedding模型

- 在线：openAI的embedding模型：准备openAI的API key、国内反向代理地址； DashScope平台（阿里云百炼）（有qwen3 embedding），准备API Key
- 本地部署开源embedding模型 Ollama官网可以搜索，ollama拉取、本地部署，例如BGE，然后可以`from langchain_ollama import OllamaEmbeddings`, 进行后续embedding。



### 4. Store: 词向量数据库

- chroma
- FAISS
- ······

### 5. Retrieve: 
根据用户的问题，在数据库中搜索最相关的文本块。

### 6. Augment: 
将检索到的文本块作为“参考背景”塞进 Prompt 模板中。

### 7. Generate: 
将组装好的 Prompt 发给 LLM，输出最终答案。


# LangChain中的RAG相关


## Document loaders

https://docs.langchain.com/oss/python/integrations/document_loaders

### 接口 (Interface)
每个文档加载器可能定义自己的参数，但它们共享一个通用的 API：

- load(): 一次性加载所有文档。
- lazy_load(): 懒加载（流式加载）文档，适用于大型数据集。

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(
    ...  # 这里填写特定集成的参数
)

# 加载所有文档
documents = loader.load()

# 对于大型数据集，延迟加载文档
for document in loader.lazy_load():
    print(document)

### 各种类型的loader（By category）

#### PyPDF

In [4]:
!pip install -qU langchain-community pypdf

1 初始化与加载 (Initialization & Load)

In [5]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "./刑法.pdf"
loader = PyPDFLoader(file_path)

# 执行加载
docs = loader.load()

# 查看第一页内容
print(docs[0].page_content)
# 查看元数据（如作者、页码、总页数等）
print(docs[0].metadata)

－1－
中华人民共和国刑法
（1979 年 7 月 1 日第五届全国人民代表大会第二次会
议通过 1997 年 3 月 14 日第八届全国人民代表大会
第五次会议修订 根据 1998 年 12 月 29 日第九届全
国人民代表大会常务委员会第六次会议通过的《全国人
民代表大会常务委员会关于惩治骗购外汇、逃汇和非法
买卖外汇犯罪的决定》、1999 年 12 月 25 日第九届全
国人民代表大会常务委员会第十三次会议通过的《中华
人民共和国刑法修正案》、2001 年 8 月 31 日第九届
全国人民代表大会常务委员会第二十三次会议通过的
《中华人民共和国刑法修正案（二）》、2001 年 12
月 29 日第九届全国人民代表大会常务委员会第二十五
次会议通过的《中华人民共和国刑法修正案（三）》、
2002 年 12 月 28 日第九届全国人民代表大会常务委员
会第三十一次会议通过的《中华人民共和国刑法修正案
（四）》、2005 年 2 月 28 日第十届全国人民代表大
会常务委员会第十四次会议通过的《中华人民共和国刑
法修正案（五）》、2006 年 6 月 29 日第十届全国人
民代表大会常务委员会第二十二次会议通过的《中华人
{'producer': '', 'creator': 'WPS 文字', 'creationdate': '2024-12-16T10:50:30+02:50', 'author': 'YF-INT6', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2024-12-16T10:50:30+02:50', 'sourcemodified': "D:20241216105030+02'50'", 'subject': '', 'title': '', 'trapped': '/False', 'source': './刑法.pdf', 'total_pages': 153, 'page': 0, 'page_label': '1'}


2 懒加载 (Lazy Load):对于大型 PDF，可以使用 lazy_load 逐页处理，以节省内存：

In [6]:
pages = []
for doc in loader.lazy_load():
    pages.append(doc)
    if len(pages) >= 10:
        # 执行某些分页操作，例如存入向量数据库
        # index.upsert(pages)
        pages = []

3 拆分模式与自定义页码分隔符
加载 PDF 时有两种模式：
- 按页拆分 (mode="page"): 每一页变为一个独立的 LangChain Document 对象。这是默认行为，元数据会包含 page 编号。
- 单文本流 (mode="single"): 整个 PDF 作为一个文档。这可以避免段落因分页而被切断。

使用自定义分隔符
在 single 模式下，你可以添加分隔符（如 \f 或自定义文字）来标记原页面结束的位置：

In [8]:
loader = PyPDFLoader(
    "./刑法.pdf",
    mode="single",
    pages_delimiter="\n-------这是自定义页末标记-------\n",
)

4 从 PDF 中提取图片 (Extract images)

PyPDFLoader 支持三种提取并识别图片中内容的方案：

    RapidOCR: 轻量级 OCR 工具（主要针对中英文）。

    Tesseract: 高精度 OCR 工具。

    多模态大模型 (LLM): 例如使用 gpt-4o 识别图片内容。

示例：使用多模态模型识别图片

In [ ]:
from langchain_community.document_loaders.parsers import LLMImageBlobParser
from langchain_openai import ChatOpenAI

loader = PyPDFLoader(
    "./刑法.pdf",
    mode="page",
    images_inner_format="markdown-img", # 输出格式
    images_parser=LLMImageBlobParser(model=ChatOpenAI(model="gpt-4o")),
)
docs = loader.load()

5 🚗高级用法：解析器分离 (Working with files)

LangChain 支持将“如何加载文件”和“如何解析内容”解耦。这让你能用同样的逻辑处理本地文件或云端文件（S3, Azure, GCS）。



## Text splitters
https://docs.langchain.com/oss/python/integrations/splitters

In [1]:
!pip install -U langchain-text-splitters

### RecursiveCharacterTextSplitter
对于大多数用例，请从 `RecursiveCharacterTextSplitter`（递归字符文本分割器） 开始。它在保持上下文完整性和管理分块大小之间提供了良好的平衡。 这种默认策略开箱即用，效果良好；只有当你需要为特定应用精细调整性能时，才应考虑调整它。

### 基于文本结构 (Text structure-based)
文本自然地组织在段落、句子和单词等层次单位中。我们可以利用这种固有的结构来指导拆分策略，从而创建能够保持自然语言流、维护分块内语义一致性并适应不同层级文本粒度的分块。 LangChain 的 RecursiveCharacterTextSplitter 实现了这一概念：

- RecursiveCharacterTextSplitter 尝试保持较大的单位（如段落）完整。
- 如果一个单位超过了分块大小（chunk size），它会移动到下一个层级（如句子）。
- 如有必要，此过程会一直持续到单词层级。

In [10]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. 加载 PDF
file_path = "./刑法.pdf"
loader = PyPDFLoader(file_path)
documents = loader.load() # 得到的是 List[Document]

# 2. 初始化分割器
# 提示：对于法律条文，建议设置 chunk_overlap（重叠度），防止条文被切断时丢失上下文
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,   # 法律条文通常较长，100 可能太小了
    chunk_overlap=50  # 前后块重叠 50 字，保证语意连贯
)

# 3. 执行分割（注意方法名是 split_documents）
# texts = text_splitter.split_text(documents)
texts = text_splitter.split_documents(documents)

# 查看分割后的结果
print(f"分割成了 {len(texts)} 个文本块")
print(f"第一个块的内容：\n{texts[0].page_content}")
print(f"第一个块的元数据：\n{texts[0].metadata}")

分割成了 243 个文本块
第一个块的内容：
－1－
中华人民共和国刑法
（1979 年 7 月 1 日第五届全国人民代表大会第二次会
议通过 1997 年 3 月 14 日第八届全国人民代表大会
第五次会议修订 根据 1998 年 12 月 29 日第九届全
国人民代表大会常务委员会第六次会议通过的《全国人
民代表大会常务委员会关于惩治骗购外汇、逃汇和非法
买卖外汇犯罪的决定》、1999 年 12 月 25 日第九届全
国人民代表大会常务委员会第十三次会议通过的《中华
人民共和国刑法修正案》、2001 年 8 月 31 日第九届
全国人民代表大会常务委员会第二十三次会议通过的
《中华人民共和国刑法修正案（二）》、2001 年 12
月 29 日第九届全国人民代表大会常务委员会第二十五
次会议通过的《中华人民共和国刑法修正案（三）》、
2002 年 12 月 28 日第九届全国人民代表大会常务委员
会第三十一次会议通过的《中华人民共和国刑法修正案
（四）》、2005 年 2 月 28 日第十届全国人民代表大
会常务委员会第十四次会议通过的《中华人民共和国刑
法修正案（五）》、2006 年 6 月 29 日第十届全国人
第一个块的元数据：
{'producer': '', 'creator': 'WPS 文字', 'creationdate': '2024-12-16T10:50:30+02:50', 'author': 'YF-INT6', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2024-12-16T10:50:30+02:50', 'sourcemodified': "D:20241216105030+02'50'", 'subject': '', 'title': '', 'trapped': '/False', 'source': './刑法.pdf', 'total_pages': 153, 'page': 0, 'page_label': '1'}


### 基于长度 (Length-based)
- 基于 Token： 根据 Token 数量拆分文本，这在处理语言模型时非常有用。
- 基于字符： 根据字符数量拆分文本，这在不同类型的文本之间可能更具一致性。

In [11]:
from langchain_text_splitters import CharacterTextSplitter
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    encoding_name="cl100k_base", chunk_size=500, chunk_overlap=50
)
# texts = text_splitter.split_text(document)
texts = text_splitter.split_documents(document)

In [12]:
texts

[Document(metadata={'producer': '', 'creator': 'WPS 文字', 'creationdate': '2024-12-16T10:50:30+02:50', 'author': 'YF-INT6', 'comments': '', 'company': '', 'keywords': '', 'moddate': '2024-12-16T10:50:30+02:50', 'sourcemodified': "D:20241216105030+02'50'", 'subject': '', 'title': '', 'trapped': '/False', 'source': './刑法.pdf', 'total_pages': 153, 'page': 0, 'page_label': '1'}, page_content='－1－\n中华人民共和国刑法\n（1979 年 7 月 1 日第五届全国人民代表大会第二次会\n议通过 1997 年 3 月 14 日第八届全国人民代表大会\n第五次会议修订 根据 1998 年 12 月 29 日第九届全\n国人民代表大会常务委员会第六次会议通过的《全国人\n民代表大会常务委员会关于惩治骗购外汇、逃汇和非法\n买卖外汇犯罪的决定》、1999 年 12 月 25 日第九届全\n国人民代表大会常务委员会第十三次会议通过的《中华\n人民共和国刑法修正案》、2001 年 8 月 31 日第九届\n全国人民代表大会常务委员会第二十三次会议通过的\n《中华人民共和国刑法修正案（二）》、2001 年 12\n月 29 日第九届全国人民代表大会常务委员会第二十五\n次会议通过的《中华人民共和国刑法修正案（三）》、\n2002 年 12 月 28 日第九届全国人民代表大会常务委员\n会第三十一次会议通过的《中华人民共和国刑法修正案\n（四）》、2005 年 2 月 28 日第十届全国人民代表大\n会常务委员会第十四次会议通过的《中华人民共和国刑\n法修正案（五）》、2006 年 6 月 29 日第十届全国人\n民代表大会常务委员会第二十二次会议通过的《中华人'),
 Document(metadata={'producer': '', 'creator': 'WPS 文字', 'creationda

### 基于文档结构 (Document structure-based)
某些文档具有固有的结构，例如 HTML、Markdown 或 JSON 文件

- Markdown： 根据标题拆分（例如 #, ##, ###）：https://docs.langchain.com/oss/python/integrations/splitters/markdown_header_metadata_splitter
- HTML： 使用标签拆分
- JSON： 按对象或数组元素拆分
- 代码： 按函数、类或逻辑块拆分

## Embedding models

LangChain 目前不支持多模态嵌入

1. 工作原理 (How it works)

- 向量化 (Vectorization)：模型将每个输入字符串编码为一个高维向量。

- 相似度评分 (Similarity scoring)：使用数学指标比较向量，以衡量底层文本的相关程度。

2. 相似度指标 (Similarity metrics)

通常使用以下几种指标来比较嵌入向量：

- 余弦相似度 (Cosine similarity)：衡量两个向量之间的夹角。

- 欧几里得距离 (Euclidean distance)：衡量点与点之间的直线距离。

- 点积 (Dot product)：衡量一个向量在另一个向量上的投影程度。

### 接口 (Interface)

LangChain 通过 Embeddings 接口为各种文本嵌入模型（如 OpenAI、Cohere、Hugging Face）提供标准接口。 主要提供两个方法：

- embed_documents(texts: List[str]): 嵌入一组文档。

- embed_query(text: str): 嵌入单个查询。

1 OpenAIEmbeddings：https://docs.langchain.com/oss/python/integrations/text_embedding/openai

In [16]:
from dotenv import load_dotenv
load_dotenv(dotenv_path="/home/crq/llm_scripts/langchain/.env")

True

2 实例化 (Instantiation)

In [24]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    base_url="https://api.chatanywhere.tech"
    # 对于 text-embedding-3 系列模型，
    # 你可以指定希望返回的向量维度。
    # dimensions=1024
)

3 索引与检索 (Indexing and retrieval)

下面展示如何使用我们上面初始化的 embeddings 对象在 InMemoryVectorStore（内存向量数据库）中索引和检索数据。

In [25]:
from langchain_core.vectorstores import InMemoryVectorStore

# 使用示例文本创建向量数据库
text = "LangChain 是构建具有上下文感知能力的推理应用的框架"
vectorstore = InMemoryVectorStore.from_texts(
    [text],
    embedding=embeddings,
)

# 将向量数据库用作检索器 (Retriever)
retriever = vectorstore.as_retriever()

# 检索最相关的文本
retrieved_documents = retriever.invoke("什么是 LangChain?")

# 显示检索到的文档内容
print(retrieved_documents[0].page_content)

LangChain 是构建具有上下文感知能力的推理应用的框架


4 直接使用 (Direct usage)

在底层，向量数据库和检索器的实现分别调用了 embeddings.embed_documents(...) 和 embeddings.embed_query(...)。你也可以直接调用这些方法：

 4.1 嵌入单条文本:embed_query

In [28]:
single_vector = embeddings.embed_query(text)
print(str(single_vector)[:100])  # 显示向量的前 100 个字符

[-0.012340659275650978, 0.0007904645754024386, -0.02307773008942604, -5.198453891352983e-06, 0.01553


4.2 嵌入多条文本：embed_documents

In [29]:
text2 = "LangGraph 是一个用于构建带状态的多角色 LLM 应用的库"
two_vectors = embeddings.embed_documents([text, text2])
for vector in two_vectors:
    print(str(vector)[:100])

[-0.012340659275650978, 0.0007904645754024386, -0.02307773008942604, -5.198453891352983e-06, 0.01553
[-0.01334487646818161, 0.01652354747056961, -0.029722994193434715, 0.004823407623916864, 0.013213298


### 缓存 (Caching)

嵌入结果可以被存储或临时缓存，以避免重复计算。 可以使用 CacheBackedEmbeddings 来缓存嵌入。这个封装器将嵌入存储在键值对存储（key-value store）中，文本会被哈希处理并作为缓存的键。

初始化 CacheBackedEmbeddings 的主要支持方式是 from_bytes_store。它包含以下参数：
- underlying_embedder: 用于生成嵌入的底层嵌入器。
- document_embedding_cache: 用于缓存文档嵌入的 ByteStore。
- namespace: (可选) 用于文档缓存的命名空间（例如使用模型名称），以避免冲突。

In [32]:
import time
from langchain_classic.embeddings import CacheBackedEmbeddings  
from langchain_classic.storage import LocalFileStore 
from langchain_core.vectorstores import InMemoryVectorStore

# Create your underlying embeddings model
# underlying_embeddings = ... # e.g., OpenAIEmbeddings(), HuggingFaceEmbeddings(), etc.
underlying_embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    base_url="https://api.chatanywhere.tech"
    # 对于 text-embedding-3 系列模型，
    # 你可以指定希望返回的向量维度。
    # dimensions=1024
)

# Store persists embeddings to the local filesystem
# This isn't for production use, but is useful for local
store = LocalFileStore("./cache/") 

cached_embedder = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings,
    store,
    namespace=underlying_embeddings.model
)

# Example: caching a query embedding
tic = time.time()
print(cached_embedder.embed_query("Hello, world!"))
print(f"First call took: {time.time() - tic:.2f} seconds")

# Subsequent calls use the cache
tic = time.time()
print(cached_embedder.embed_query("Hello, world!"))
print(f"Second call took: {time.time() - tic:.2f} seconds")

/home/crq/.conda/envs/langchain/lib/python3.11/site-packages/langchain_classic/embeddings/cache.py:58: UserWarning: Using default key encoder: SHA-1 is *not* collision-resistant. While acceptable for most cache scenarios, a motivated attacker can craft two different payloads that map to the same cache key. If that risk matters in your environment, supply a stronger encoder (e.g. SHA-256 or BLAKE2) via the `key_encoder` argument. If you change the key encoder, consider also creating a new cache, to avoid (the potential for) collisions with existing keys.
  _warn_about_sha1_encoder()


[-0.005865450482815504, -0.024112658575177193, -0.022020621225237846, 0.027614882215857506, -0.002742892364040017, -0.02045546844601631, -0.0026905913837254047, 0.05324620380997658, 0.025057949125766754, 0.023787230253219604, -0.004071722738444805, 0.017511120066046715, 0.003260090248659253, -0.0046605924144387245, -0.0026654095854610205, 0.03359655663371086, -0.033038679510354996, 0.016162918880581856, 0.0018799272365868092, -0.02969142235815525, 0.03043525665998459, -0.00902674999088049, -0.035239193588495255, 0.01238175667822361, 0.008546357043087482, 0.024050671607255936, 0.0014905760763213038, 0.0034305525477975607, -0.02825024165213108, -0.030807174742221832, 0.003953561652451754, 0.006926965434104204, 0.008089208044111729, 0.04648970067501068, 0.04642771556973457, 0.006907594855874777, 0.006760377436876297, 0.03217087313532829, 0.02432961016893387, 0.012575464323163033, 0.008104704320430756, 0.03852446377277374, -0.009739592671394348, 0.02620469406247139, 0.005187475588172674, 0

在生产环境中，通常会使用更强大的持久化存储，例如数据库或云存储。

## Vector stores

https://docs.langchain.com/oss/python/integrations/vectorstores

In [36]:
!pip install langchain_chroma

  Using cached pybase64-1.4.3-cp311-cp311-manylinux1_x86_64.manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_5_x86_64.whl.metadata (8.7 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached onnxruntime-1.23.2-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (5.1 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.39.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached tokenizers-0.22.2-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.3 kB)
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached bcrypt-5.0.0-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (10 kB)
  Using cached mmh3-5.2.0-cp311-cp311-manylinux1_x86_64.manylinux_2_28_x86_64.manylinux_2_5_x86_64.whl.metadata (14 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl.metadata (1.3 

In [48]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# 1. 加载并分割
loader = PyPDFLoader("./刑法.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(documents)

# 2. 初始化嵌入模型
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", base_url="https://api.chatanywhere.tech")

# 3. 创建并存入向量数据库 (存入本地 'chroma_db' 目录)
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=embeddings,
    persist_directory="./chroma_db"
)

print("刑法条文已成功存入向量数据库！")

刑法条文已成功存入向量数据库！


In [44]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

# 1. 加载所有页面
loader = PyPDFLoader("./刑法.pdf")
all_documents = loader.load()

# --- 关键修改：只保留前 10 页 ---
# all_documents 是一个列表，列表里的每个元素代表一页
documents_10_pages = all_documents[:20] 
# ------------------------------

# 2. 对前 10 页进行分割
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(documents_10_pages)

# 3. 初始化嵌入模型
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", base_url="https://api.chatanywhere.tech")
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
print("###########################################")

# 4. 存入向量数据库
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=embeddings,
    persist_directory="./chroma_db"
)

print(f"成功将前 {len(documents_10_pages)} 页刑法条文存入向量数据库！")

###########################################
成功将前 20 页刑法条文存入向量数据库！


## Retrievers

https://docs.langchain.com/oss/python/integrations/retrievers

刚才已经把前 10 页《刑法》存入了 Chroma。现在，我们只需要一行代码就能把它变成一个“检索器”，并开始提问：

In [47]:
# 1. 将向量数据库转换为检索器
retriever = vectorstore.as_retriever(
    search_kwargs={"k": 1} # 每次检索返回最相关的 3 个法条块
)

# 2. 提出一个法律问题
# query = "精神病人违法怎么处理？"
query = "什么是犯罪未遂？"
retrieved_docs = retriever.invoke(query)

# 3. 查看结果
for i, doc in enumerate(retrieved_docs):
    print(f"--- 匹配结果 {i+1} (来自第 {doc.metadata['page']+1} 页) ---")
    print(doc.page_content)

--- 匹配结果 1 (来自第 10 页) ---
－10－
财产和其他权利免受正在发生的危险，不得已采取的紧急避险行
为，造成损害的，不负刑事责任。
紧急避险超过必要限度造成不应有的损害的，应当负刑事责
任，但是应当减轻或者免除处罚。
第一款中关于避免本人危险的规定，不适用于职务上、业务
上负有特定责任的人。
第二节 犯罪的预备、未遂和中止
第二十二条 为了犯罪，准备工具、制造条件的，是犯罪预
备。
对于预备犯，可以比照既遂犯从轻、减轻处罚或者免除处罚。
第二十三条 已经着手实行犯罪，由于犯罪分子意志以外的
原因而未得逞的，是犯罪未遂。
对于未遂犯，可以比照既遂犯从轻或者减轻处罚。
第二十四条 在犯罪过程中，自动放弃犯罪或者自动有效地
防止犯罪结果发生的，是犯罪中止。
对于中止犯，没有造成损害的，应当免除处罚；造成损害的，
应当减轻处罚。
第三节 共同犯罪


# GraphRAG

## GraphRAG原理

### 索引阶段

文本切分 ➡️ 实体识别 ➡️ 关系挖掘 ➡️  文本嵌入 ➡️ 构建社区和层级结构 ➡️ 生成索引文件

在GraphRAG的索引过程中，实体识别是与文本切分和关系挖掘同时进行的。具体来说：​

1. 实体识别：GraphRAG使用大语言模型（如GPT）识别文本中的实体，并根据其类型为每个实体赋予一个唯一标识符。​

2. 实体嵌入：GraphRAG为每个实体生成语义嵌入，即一个高维向量，来描述该实体的语义。实体嵌入可以帮助模型理解实体在不同上下文中的意义和关系。​

3. 关系挖掘：在识别实体之后，GraphRAG还会识别实体之间的关系，例如“作者是”、“属于”等。这些关系构成了知识图谱中的边，实体则是图谱中的节点。

关系挖掘中，Level 和 Degree 是描述节点在图结构中“位置”和“影响力”的两个核心维度。


| 维度 | 核心概念一：度数 (Degree) | 核心概念二：等级 (Level) |
| :--- | :--- | :--- |
| **定义** | 节点直接相连的**边数量**。 | 实体在图谱中的**层次、深度或核心性**。 |
| **侧重点** | 衡量实体的**连接性**与局部活跃度。 | 衡量实体的**全局重要性**或所属结构。 |
| **细分指标** | 1. **入度 (In-degree)**：指向该节点的边。<br>2. **出度 (Out-degree)**：由该节点发出的边。 | 1. **中心性算法**：如 PageRank、近邻中心性。<br>2. **社区检测**：划分节点所属的群体。 |
| **核心逻辑** | 度数越高，实体越活跃，关系强度越大。 | 得分越高，实体越处于网络的核心地位。 |
| **应用场景** | 统计实体的直接关联数量，识别“显性”热门节点。 | 识别“隐性”权威节点或复杂的组织结构关系。 |


在知识图谱和网络科学中，社区检测是指将图中的节点划分为若干个组（即“社区”），使得同一个组内的节点连接非常紧密，而不同组之间的节点连接相对稀疏。

    Louvain 算法

### 查询阶段

GraphRAG开源项目：

|项目名称|​描述|
|:---|:---|
| Microsoft GraphRAG | 一个能够将知识图谱和 RAG 结合起来的数据工作流和转换工具，可以提供数据处理，检索问答能力 |
|RAGFlow|基于深度文档理解构建的开源 RAG（Retrieval-Augmented Generation）引擎​|
|kotaemon|集成 GraphRAG 及混合检索等方法的具备本地知识库问答的项目​|

### GraphRAG缺点：

| 维度 | 缺点描述 | 影响后果 |
| :--- | :--- | :--- |
| **经济成本** | **极高的 Token 消耗** | 构建索引阶段需多次调用 LLM 进行实体提取、关系映射及社区摘要生成，其 API 费用通常是传统向量 RAG 的 **50-100 倍**。 |
| **时间效率** | **索引构建周期长** | 知识图谱的构建涉及大量计算与逻辑推理。传统 RAG 解析文档以分钟计，而 GraphRAG 处理同规模文档可能需要数小时。 |
| **查询延迟** | **端到端响应高延迟** | 在执行全局查询 (Global Search) 时，系统需实时聚合多个社区摘要。用户等待回答的时间往往从毫秒级跳升至 **10-60 秒**。 |
| **提取质量** | **高度依赖底层模型** | 知识图谱的准确性完全取决于 LLM 的指令遵循能力。若模型稍弱，生成的图谱会出现大量“虚假关联”或“断裂节点”。 |
| **系统复杂性** | **运维与开发门槛高** | 需要同时维护向量数据库与**图数据库**（如 Neo4j）。技术栈更复杂，且存在大量如“社区发现层次”等难以调优的参数。 |
| **数据动态性** | **增量更新维护成本高** | 当源文档发生变动时，往往需要重新计算受影响的图结构及其上层的所有摘要，无法像传统 RAG 那样实现秒级实时入库。 |
| **解析噪音** | **实体歧义与图漂移** | 在处理同名实体时（如“张三”指代不同人），若实体消解失败，会导致图谱逻辑链条断裂，甚至产生严重的语义噪音。 |



## GraphRAG入门

# 垂直领域 Agentic-GraphRAG 开发实战​

 考虑到GraphRAG方案的复杂性，我们在垂直领域应用中可以采用更务实的实现方式。一个思路是借助大语言模型的Agent能力，结合知识图谱思路，实现Agentic-GraphRAG的效果。这也是本课程的核心内容：不是手动构建庞大的图数据库和复杂检索算法，而是利用 LangChain 等框架中现有组件，把知识检索、关系推理融合在智能 Agent 的决策过程中。

## 非结构化数据清洗：OCR 与 LangExtract 技术方案拆解

### MinerU: https://github.com/opendatalab/MinerU

MinerU 项目非常适用于：​
- 需要从大量 PDF 文档中抽取结构化内容（例如学术文献、技术白皮书、报告）用于知识库或训练语料。​
- 对版式结构（如章节、列表、表格、公式）要求较高，而不只是 OCR 文本识别。​
- 希望输出 Markdown／JSON 供后续自动化流水线使用。

PaddleOCR:https://github.com/PaddlePaddle/PaddleOCR/tree/main

DeepSeek-OCR:https://github.com/deepseek-ai/DeepSeek-OCR

dots.ocr

### LangExtract信息抽取

LangExtract 是 Google 开源的一个 Python 库,它的核心使命可以用一句话概括:利用大型语言模型,将非结构化文本转换为结构化数据,并确保每个提取结果都精确对应回原文位置。

- 能力一:零代码定义提取任务(Prompt-Driven)​

```bash
    # 示例:定义提取任务的 Prompt​
    extraction_prompt = """​
    从临床笔记中提取以下信息:​
    1. 药物名称(medication)​
    2. 剂量(dosage) ​
    3. 给药频率(frequency)​
    4. 持续时间(duration)​
​
    请确保提取的内容与原文完全一致。​
    """​
​
    # 这就是全部!无需编写任何解析逻辑
```

- 能力二:少样本学习(Few-Shot Learning)​

  仅有 prompt 还不够,LangExtract 支持提供少量示例来"教会"模型输出的具体格式:​

```bash
​
    import langextract as lx​
​
    # 定义一个示例:告诉模型"什么样的输入应该产生什么样的输出"​
    example = lx.data.ExampleData(​
        text="患者目前服用缬沙坦80mg每日一次,已持续3个月。",​
        extractions=[​
            lx.data.Extraction(​
                text="缬沙坦",​
                class_name="medication"​
            ),​
            lx.data.Extraction(​
                text="80mg",​
                class_name="dosage"​
            ),​
            lx.data.Extraction(​
                text="每日一次",​
                class_name="frequency"​
            ),​
            lx.data.Extraction(​
                text="3个月",​
                class_name="duration"​
            )​
        ]​
    )​
​
    # 提供1-3个这样的示例,模型就能学会输出格式!
```

- 能力三:精确的来源定位(Source Grounding)​

每个提取结果都会自动标注其在原文中的字符偏移位置。

- 能力四:结构化输出保证(Schema Enforcement)

- 能力五:长文档多轮提取(Multi-Pass Extraction)​


## LangExtract 长文本提取实战

In [10]:
!pip install langextract

  Using cached absl_py-2.3.1-py3-none-any.whl.metadata (3.3 kB)
  Using cached async_timeout-5.0.1-py3-none-any.whl.metadata (5.1 kB)
Using cached absl_py-2.3.1-py3-none-any.whl (135 kB)
Using cached async_timeout-5.0.1-py3-none-any.whl (6.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [langextract] [langextract]


In [13]:
import os
import textwrap
from pathlib import Path
from collections import Counter, defaultdict

from dotenv import load_dotenv
import langextract as lx
from langextract.providers.openai import OpenAILanguageModel
from langextract.prompt_validation import PromptValidationLevel

print("所有依赖库导入成功！")

# 加载环境变量​
load_dotenv(dotenv_path="/home/crq/llm_scripts/langchain/.env")

所有依赖库导入成功！


True

In [14]:
# 配置 DeepSeek API
DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY", "")
DASHSCOPE_BASE_URL = os.getenv("DASHSCOPE_BASE_URL", "https://api.deepseek.com")
DEFAULT_MODEL = os.getenv("DEFAULT_MODEL", "qwen3-max")

# 配置输出目录
OUTPUT_DIR = Path("./outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

print(f"配置完成！")
print(f"  模型: {DEFAULT_MODEL}")
print(f"  API 端点: {DASHSCOPE_BASE_URL}")
print(f"  输出目录: {OUTPUT_DIR}")

配置完成！
  模型: qwen3-max
  API 端点: https://dashscope.aliyuncs.com/compatible-mode/v1
  输出目录: outputs


读取长文本数据

In [16]:
# 读取《罗密欧与朱丽叶》中文文本
romeo_file = Path("/home/crq/llm_scripts/rag/data/罗密欧与朱丽叶_part.txt")

if romeo_file.exists():
    # 尝试多种编码
    for encoding in ['gbk', 'gb2312', 'utf-8', 'utf-16']:
        try:
            with open(romeo_file, 'r', encoding=encoding) as f:
                input_text = f.read()
            print(f"✓ 成功使用 {encoding} 编码读取文件")
            break
        except:
            continue
else:
    print("错误: 未找到 罗密欧与朱丽叶.txt 文件")
    print("请确保文件存在于当前目录")
    input_text = ""  # 防止后面 len(input_text) 报错

if input_text:
    print(f"✓ 成功读取 {len(input_text):,} 字符")
    print(f"\n文本预览（前 200 字符）：")
    print("-" * 60)
    print(input_text[:200])
    print("...")

✓ 成功使用 utf-8 编码读取文件
✓ 成功读取 1,592 字符

文本预览（前 200 字符）：
------------------------------------------------------------
『罗密欧与朱丽叶/作者:莎士比亚』
『状态:已完结』
『内容简介:
内容简介：
　　 《罗密欧与朱丽叶》，威廉&middot;莎士比亚著名戏剧作品之一，故事讲述二人于舞会一见钟情后方知对方身份，最后二人为了在一起，朱丽叶先服假毒，醒来发现罗密欧自尽，也相继自尽。因其知名度而常被误称为莎翁四大悲剧之一。』
                                 
第一幕
    第一场维洛
...


定义提取任务 Prompt

In [18]:
import textwrap

# 定义提取任务的 Prompt 描述
prompt = textwrap.dedent("""\
    从中文戏剧文本中提取人物、情感和关系信息。
    
    为每个实体添加有意义的属性以增加上下文。
    
    重要: extraction_text 必须是原文的精确子串，不要改写。
    按出现顺序提取实体，不要重叠。
    
    注意: 在戏剧脚本中，角色名称后跟冒号。""")

print("Prompt 定义完成！")
print("\n" + "=" * 60)
print("Prompt 内容：")
print("=" * 60)
print(prompt)

Prompt 定义完成！

Prompt 内容：
从中文戏剧文本中提取人物、情感和关系信息。

为每个实体添加有意义的属性以增加上下文。

重要: extraction_text 必须是原文的精确子串，不要改写。
按出现顺序提取实体，不要重叠。

注意: 在戏剧脚本中，角色名称后跟冒号。


定义 Few-shot 示例

In [19]:
examples = [
    lx.data.ExampleData(
        text=textwrap.dedent("""\
            罗密欧: 轻声！那边窗子里亮起来的是什么光？
            那就是东方，朱丽叶就是太阳。
            朱丽叶: 啊！罗密欧，罗密欧！为什么你偏偏是罗密欧呢？"""),
        extractions=[
            lx.data.Extraction(
                extraction_class="人物",
                extraction_text="罗密欧",
                attributes={"情感状态": "惊叹"}
            ),
            lx.data.Extraction(
                extraction_class="情感",
                extraction_text="轻声！",
                attributes={"情感": "温柔敬畏", "人物": "罗密欧"}
            ),
            lx.data.Extraction(
                extraction_class="关系",
                extraction_text="朱丽叶就是太阳",
                attributes={"类型": "比喻", "人物1": "罗密欧", "人物2": "朱丽叶"}
            ),
            lx.data.Extraction(
                extraction_class="人物",
                extraction_text="朱丽叶",
                attributes={"情感状态": "渴望"}
            ),
            lx.data.Extraction(
                extraction_class="情感",
                extraction_text="为什么你偏偏是罗密欧呢？",
                attributes={"情感": "渴望的疑问", "人物": "朱丽叶"}
            ),
        ]
    )
]

print("Few-shot 示例定义完成！")
print(f"\n示例数量: {len(examples)}")
print(f"示例文本长度: {len(examples[0].text)} 字符")
print(f"示例提取数量: {len(examples[0].extractions)} 个")
print("\n" + "=" * 60)
print("示例文本：")
print("=" * 60)
print(examples[0].text)

Few-shot 示例定义完成！

示例数量: 1
示例文本长度: 65 字符
示例提取数量: 5 个

示例文本：
罗密欧: 轻声！那边窗子里亮起来的是什么光？
那就是东方，朱丽叶就是太阳。
朱丽叶: 啊！罗密欧，罗密欧！为什么你偏偏是罗密欧呢？


 创建 LangExtract 模型

In [20]:
# 创建 DeepSeek 模型实例
model = OpenAILanguageModel(
    model_id=DEFAULT_MODEL,
    api_key=DASHSCOPE_API_KEY,
    base_url=DASHSCOPE_BASE_URL
)

print("模型创建成功！")
print(f"  模型 ID: {DEFAULT_MODEL}")
print(f"  API 端点: {DASHSCOPE_BASE_URL}")

模型创建成功！
  模型 ID: qwen3-max
  API 端点: https://dashscope.aliyuncs.com/compatible-mode/v1


 运行长文本提取（核心）

In [21]:
# 运行长文本提取
print("开始运行长文本提取...")
print(f"   输入文本: {len(input_text):,} 字符")
print(f"   提取轮数: 3 轮")
print(f"   并行数: 20 workers")
print(f"   分块大小: 1000 字符")
print("\n正在处理，请稍候...\n")

result = lx.extract(
    text_or_documents=input_text,
    prompt_description=prompt,
    examples=examples,
    model=model,
    extraction_passes=3,      # 多轮提取提高召回率
    max_workers=20,           # 并行处理加速
    max_char_buffer=1000      # 较小的上下文提高准确性
)

print("\n" + "=" * 60)
print("提取完成！")
print("=" * 60)
print(f"文本长度: {len(result.text):,} 字符")
print(f"提取实体总数: {len(result.extractions)} 个")

/home/crq/.conda/envs/langchain/lib/python3.11/site-packages/langextract/extraction.py:199: UserWarning: batch_length (10) < max_workers (20). Only 10 workers will be used. Set batch_length >= max_workers for optimal parallelization.
  warnings.warn(
/home/crq/.conda/envs/langchain/lib/python3.11/site-packages/langextract/__init__.py:55: UserWarning: 'use_schema_constraints' is ignored when 'model' is provided. The model should already be configured with schema constraints.
  return extract_func(*args, **kwargs)


开始运行长文本提取...
   输入文本: 1,592 字符
   提取轮数: 3 轮
   并行数: 20 workers
   分块大小: 1000 字符

正在处理，请稍候...



LangExtract: model=qwen3-max, current=1,585 chars, processed=0 chars:  [00:22]



提取完成！
文本长度: 1,592 字符
提取实体总数: 35 个


保存提取结果

In [22]:
# 保存提取结果为 JSONL 格式
lx.io.save_annotated_documents(
    [result], 
    output_name="罗密欧与朱丽叶_extractions.jsonl", 
    output_dir=str(OUTPUT_DIR)
)

output_jsonl_path = OUTPUT_DIR / "罗密欧与朱丽叶_extractions.jsonl"
print(f"结果已保存: {output_jsonl_path}")

LangExtract: Saving to 罗密欧与朱丽叶_extractions.jsonl: 1 docs [00:00, 206.45 docs/s]

✓ Saved 1 documents to 罗密欧与朱丽叶_extractions.jsonl
结果已保存: outputs/罗密欧与朱丽叶_extractions.jsonl


生成交互式可视化

In [24]:
# 生成交互式可视化
html_content = lx.visualize(str(output_jsonl_path))

html_file = OUTPUT_DIR / "罗密欧与朱丽叶_visualization.html"
with open(html_file, "w", encoding="utf-8") as f:
    if hasattr(html_content, 'data'):
        f.write(html_content.data)
    else:
        f.write(html_content)

print(f"交互式可视化已生成: {html_file}")
print("\n提示: 在浏览器中打开 HTML 文件可以:")
print("   - 查看高亮显示的所有提取结果")
print("   - 按类别筛选实体")
print("   - 查看每个实体的详细属性")

LangExtract: Loading 罗密欧与朱丽叶_extractions.jsonl: 100%|██████████| 15.4k/15.4k [00:00<00:00, 23.1MB/s]

✓ Loaded 1 documents from 罗密欧与朱丽叶_extractions.jsonl
交互式可视化已生成: outputs/罗密欧与朱丽叶_visualization.html

提示: 在浏览器中打开 HTML 文件可以:
   - 查看高亮显示的所有提取结果
   - 按类别筛选实体
   - 查看每个实体的详细属性


In [26]:
html_content

# Agentic GraphRAG 构建与溯源实战

In [28]:
# 标准库导入
import os
import json
import textwrap
import time
import zipfile
import io
import uuid
from pathlib import Path
from typing import Optional
from dataclasses import dataclass, field, asdict
from collections import defaultdict, Counter

print("标准库导入成功")

标准库导入成功


In [29]:
# 加载环境变量
try:
    from dotenv import load_dotenv
    load_dotenv(dotenv_path="/home/crq/llm_scripts/langchain/.env")
    print("已加载 .env 配置文件")
except ImportError:
    print("未安装 python-dotenv，将使用系统环境变量")

已加载 .env 配置文件


In [31]:
# HTTP 请求库（用于调用 MinerU API）
import requests

# LangExtract 核心库
import langextract as lx
from langextract.providers.openai import OpenAILanguageModel
from langextract.prompt_validation import PromptValidationLevel

print("LangExtract 导入成功")
print(f"  版本信息: {lx.__version__ if hasattr(lx, '__version__') else '未知'}")

LangExtract 导入成功
  版本信息: 未知


In [32]:
%pip install chromadb langchain-chroma langchain_openai

Note: you may need to restart the kernel to use updated packages.


In [33]:
# ChromaDB 向量数据库
import chromadb
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

print("ChromaDB 和 LangChain 导入成功")

ChromaDB 和 LangChain 导入成功


In [36]:
# MinerU API 配置
MINERU_API_KEY = os.getenv("MINERU_API_KEY", "")
MINERU_BASE_URL = "https://mineru.net"

print("MinerU 配置:")
if MINERU_API_KEY:
    print(f"  API Key: {MINERU_API_KEY[:10]}... (已配置)")
    print(f"  API URL: {MINERU_BASE_URL}")
else:
    print("  API Key 未配置")

MinerU 配置:
  API Key: zlm1kwd5gl... (已配置)
  API URL: https://mineru.net


In [37]:
model = OpenAILanguageModel(
    model_id=DEFAULT_MODEL,
    api_key=DASHSCOPE_API_KEY,
    base_url=DASHSCOPE_BASE_URL
)

print("模型创建成功！")
print(f"  模型 ID: {DEFAULT_MODEL}")
print(f"  API 端点: {DASHSCOPE_BASE_URL}")

模型创建成功！
  模型 ID: qwen3-max
  API 端点: https://dashscope.aliyuncs.com/compatible-mode/v1


In [38]:
# 阿里云百炼 Embeddings 配置
DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY", "")
DASHSCOPE_BASE_URL = os.getenv("DASHSCOPE_BASE_URL", "https://dashscope.aliyuncs.com/compatible-mode/v1")
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL", "text-embedding-v3")

print("\n阿里云百炼 Embeddings 配置:")
if DASHSCOPE_API_KEY:
    print(f"  API Key: {DASHSCOPE_API_KEY[:10]}... (已配置)")
    print(f"  API URL: {DASHSCOPE_BASE_URL}")
    print(f"  Embedding 模型: {EMBEDDING_MODEL}")
else:
    print("  API Key 未配置")


阿里云百炼 Embeddings 配置:
  API Key: sk-c4d40f8... (已配置)
  API URL: https://dashscope.aliyuncs.com/compatible-mode/v1
  Embedding 模型: text-embedding-v3


## 定义 ChromaDB 和输出目录配置

In [40]:
# ChromaDB 配置
CHROMA_PERSIST_DIR = "./chroma_db_chapter5"
COLLECTION_NAME = "pdf_knowledge_graph"

# 输出目录配置
OUTPUT_DIR = Path("./outputs")
OUTPUT_DIR.mkdir(exist_ok=True)

print("\n存储配置:")
print(f"  ChromaDB 存储路径: {CHROMA_PERSIST_DIR}")
print(f"  集合名称: {COLLECTION_NAME}")
print(f"  输出目录: {OUTPUT_DIR}")


存储配置:
  ChromaDB 存储路径: ./chroma_db_chapter5
  集合名称: pdf_knowledge_graph
  输出目录: outputs


### 定义数据结构

步骤：定义 KnowledgeExtraction 数据类

In [41]:
# 定义知识提取结果的数据结构
@dataclass
class KnowledgeExtraction:
    """知识提取结果（带溯源）"""
    doc_id: str                              # 文档 ID
    doc_title: str                           # 文档标题
    extraction_class: str                    # 提取类型（实体、关系、事件等）
    extraction_text: str                     # 提取的原文文本
    char_interval: Optional[dict] = None     # 原文字符区间（溯源的核心）
    attributes: dict = field(default_factory=dict)  # 属性信息
    
    def to_dict(self) -> dict:
        """转换为字典格式"""
        return asdict(self)
    
    def to_searchable_text(self) -> str:
        """生成用于向量化的可搜索文本"""
        parts = [
            f"类型: {self.extraction_class}",
            f"内容: {self.extraction_text}",
            f"来源: {self.doc_title}"
        ]
        if self.attributes:
            for k, v in self.attributes.items():
                parts.append(f"{k}: {v}")
        return " | ".join(parts)

print("KnowledgeExtraction 数据结构定义完成")
print("\n字段说明:")
print("  - doc_id: 文档唯一标识")
print("  - doc_title: 文档名称")
print("  - extraction_class: 实体类型（如'实体'、'关系描述'、'数据指标'）")
print("  - extraction_text: 从原文提取的精确文本")
print("  - char_interval: {'start_pos': 起始位置, 'end_pos': 结束位置} ← 溯源关键")
print("  - attributes: 实体的属性信息（如类型、角色等）")

KnowledgeExtraction 数据结构定义完成

字段说明:
  - doc_id: 文档唯一标识
  - doc_title: 文档名称
  - extraction_class: 实体类型（如'实体'、'关系描述'、'数据指标'）
  - extraction_text: 从原文提取的精确文本
  - char_interval: {'start_pos': 起始位置, 'end_pos': 结束位置} ← 溯源关键
  - attributes: 实体的属性信息（如类型、角色等）


### MinerU PDF 解析

MinerU 是一个专业的 PDF 解析服务，能够：​识别文档结构（标题、段落、表格、公式）​、将 PDF 转换为结构化的 Markdown 格式​ 、保留原文的语义关系​

步骤 1：准备 PDF 文件路径

In [42]:
# 指定要解析的 PDF 文件
PDF_PATH = "/home/crq/llm_scripts/rag/data/刑法4页.pdf"  # 修改为你的 PDF 文件路径

# 检查文件是否存在
pdf_file = Path(PDF_PATH)
if pdf_file.exists():
    file_size = pdf_file.stat().st_size
    print(f"PDF 文件存在: {PDF_PATH}")
    print(f"  文件大小: ({file_size/1024:.2f} KB)")
else:
    print(f"PDF 文件不存在: {PDF_PATH}")
    print("  请修改 PDF_PATH 变量为有效的文件路径")

PDF 文件存在: /home/crq/llm_scripts/rag/data/刑法4页.pdf
  文件大小: (341.12 KB)


步骤 2：请求预签名上传 URL​

向 MinerU API 发送请求，告诉服务器我们要上传一个 PDF 文件，并获取一个临时的上传 URL 和批次 ID。

In [79]:
load_dotenv(dotenv_path="/home/crq/llm_scripts/langchain/.env", override=True)

MINERU_API_KEY = os.getenv("MINERU_API_KEY")
MINERU_API_URL = os.getenv("MINERU_API_URL")

In [71]:
# 记录开始时间
parse_start_time = time.time()

# 准备 HTTP 请求头
headers = {
    "Authorization": f"Bearer {MINERU_API_KEY}",
    "Content-Type": "application/json",
}

# 步骤1：请求预签名上传 URL
print("步骤1：请求上传 URL...")
response = requests.post(
    f"{MINERU_BASE_URL}/api/v4/file-urls/batch",
    headers=headers,
    json={
        "files": [{"name": pdf_file.name, "data_id": pdf_file.name}],
        "model_version": "vlm",  # 使用视觉语言模型版本
    }
)
response.raise_for_status()  # 如果请求失败，抛出异常

# 解析响应
data = response.json()["data"]
batch_id = data["batch_id"]
upload_url = data["file_urls"][0]

print(f"  批次 ID: {batch_id}")
print(f"  上传 URL: {upload_url[:50]}...")

步骤1：请求上传 URL...
  批次 ID: 6bf77bbc-084d-4f0a-89e1-f022d433d32d
  上传 URL: https://mineru.oss-cn-shanghai.aliyuncs.com/api-up...


步骤 3：上传 PDF 文件​


In [72]:
# 步骤2：上传 PDF 文件
print("\n步骤2：上传 PDF...")

with open(PDF_PATH, "rb") as f:
    file_content = f.read()
    upload_response = requests.put(upload_url, data=file_content)
    upload_response.raise_for_status()

print(f"   上传速度: {len(file_content) / (time.time() - parse_start_time) / 1024:.2f} KB/s")


步骤2：上传 PDF...
   上传速度: 2.52 KB/s


步骤 4：轮询等待解析完成

In [81]:
# 步骤3：轮询等待解析
print("\n步骤3：等待解析...")

max_wait = 600  # 最长等待时间（秒）
poll_interval = 3  # 轮询间隔（秒）
elapsed = 0
full_zip_url = None

while elapsed < max_wait:
    # 查询解析状态
    status_response = requests.get(
        f"{MINERU_API_URL}/api/v4/extract-results/batch/{batch_id}",
        headers={"Authorization": f"Bearer {MINERU_API_KEY}"}
    )
    result = status_response.json()["data"]["extract_result"][0]
    state = result["state"]
    
    if state == "done":
        # 解析完成
        full_zip_url = result["full_zip_url"]
        print(f"   ✓ 解析完成！(耗时 {elapsed}秒)")
        break
    elif state == "failed":
        # 解析失败
        error_msg = result.get('err_msg', '未知错误')
        print(f"   解析失败: {error_msg}")
        break
    elif state == "running":
        # 正在解析
        progress = result.get("extract_progress", {})
        extracted = progress.get("extracted_pages", 0)
        total = progress.get("total_pages", "?")
        print(f"   {extracted}/{total} 页 (已等待 {elapsed}秒)", end="\r")
    
    time.sleep(poll_interval)
    elapsed += poll_interval

if elapsed >= max_wait:
    print(f"\n   解析超时 (超过 {max_wait}秒)")


步骤3：等待解析...
   ✓ 解析完成！(耗时 0秒)


-  注意：对于复杂的 PDF（如包含大量图表、公式的学术论文），解析时间可能较长。如果经常超时，可以增加 max_wait 的值。

步骤5：下载并提取 Markdown

In [82]:
# 步骤4：下载并提取 Markdown
if full_zip_url:
    print("\n步骤4：下载解析结果...")
    
    # 下载 ZIP 包
    zip_response = requests.get(full_zip_url)
    
    # 解压并提取 Markdown 文件
    with zipfile.ZipFile(io.BytesIO(zip_response.content)) as zf:
        # 查找 .md 文件
        md_files = [f for f in zf.namelist() if f.endswith('.md')]
        
        if md_files:
            # 优先选择文件名包含 'full' 的文件，否则选第一个
            target_file = next((f for f in md_files if 'full' in f.lower()), md_files[0])
            
            # 读取 Markdown 内容
            with zf.open(target_file) as f:
                markdown_text = f.read().decode('utf-8')
            
            print(f"   成功提取 Markdown: {target_file}")
            print(f"   Markdown 长度: {len(markdown_text):,} 字符")
        else:
            print("   ZIP 包中未找到 .md 文件")
            markdown_text = None
    
    # 统计解析总耗时
    parse_time = time.time() - parse_start_time
    print(f"\nPDF 解析完成 (总耗时 {parse_time:.2f}秒)")
else:
    print("\n未能获取解析结果")
    markdown_text = None


步骤4：下载解析结果...
   成功提取 Markdown: full.md
   Markdown 长度: 1,812 字符

PDF 解析完成 (总耗时 573.61秒)


步骤 6：保存 Markdown 到文件

In [85]:
# 保存 Markdown 到文件
if markdown_text:
    md_file = OUTPUT_DIR / "parsed_document.md"
    with open(md_file, "w", encoding="utf-8") as f:
        f.write(markdown_text)
    
    print(f"Markdown 已保存: {md_file}")
    print(f"\nMarkdown 预览（前500字符）:")
    print("-" * 80)
    print(markdown_text[:500])
    print("-" * 80)
else:
    print("没有 Markdown 内容可保存")

Markdown 已保存: outputs/parsed_document.md

Markdown 预览（前500字符）:
--------------------------------------------------------------------------------
第七节 走私、贩卖、运输、制造毒品罪

第八节 组织、强迫、引诱、容留、介绍卖淫罪

第九节 制作、贩卖、传播淫秽物品罪

第七章 危害国防利益罪

第八章 贪污贿赂罪

第九章 读职罪

第十章 军人违反职责罪

附则

# 第一编 总则

# 第一章 刑法的任务、基本原则和适用范围

第一条 为了惩罚犯罪，保护人民，根据宪法，结合我国同犯罪作斗争的具体经验及实际情况，制定本法。

第二条 中华人民共和国刑法的任务，是用刑罚同一切犯罪行为作斗争，以保卫国家安全，保卫人民民主专政的政权和社会主义制度，保护国有财产和劳动群众集体所有的财产，保护公民私人所有的财产，保护公民的人身权利、民主权利和其他权利，维护社会秩序、经济秩序，保障社会主义建设事业的顺利进行。

第三条 法律明文规定为犯罪行为的，依照法律定罪处刑；法律没有明文规定为犯罪行为的，不得定罪处刑。

第四条 对任何人犯罪，在适用法律上一律平等。不允许任何人有超越法律的特权。

第五条 刑罚的轻重，应当与犯罪分子所犯罪行和承担的刑事责任相适应。

第六条 凡在中华人民共和国领域内犯罪的，除法律有特别规定的以外，都适用本法。

凡在
--------------------------------------------------------------------------------


### LangExtract 知识提取

In [86]:
# 定义知识提取的 Prompt
extraction_prompt = textwrap.dedent("""
    从文档中提取以下结构化知识:
    
    - 实体: 人物、机构、地点、时间、概念、技术术语
    - 数据指标: 数值、百分比、统计数据
    - 关系描述: 实体之间的关系（合作、隶属、引用等）
    - 事件: 重要事件和行为
    
    要求:
    1. extraction_text 必须是原文的精确子串
    2. 为每个提取添加丰富的属性信息
    3. 关系类型必须在 attributes 中标注涉及的主体
    4. 保持原文出现顺序
    """)

print("提取 Prompt 定义完成")
print("\nPrompt 内容:")
print(extraction_prompt)

提取 Prompt 定义完成

Prompt 内容:

从文档中提取以下结构化知识:

- 实体: 人物、机构、地点、时间、概念、技术术语
- 数据指标: 数值、百分比、统计数据
- 关系描述: 实体之间的关系（合作、隶属、引用等）
- 事件: 重要事件和行为

要求:
1. extraction_text 必须是原文的精确子串
2. 为每个提取添加丰富的属性信息
3. 关系类型必须在 attributes 中标注涉及的主体
4. 保持原文出现顺序



定义 Few-shot 示例

In [87]:
# 定义 Few-shot 示例
example_text = textwrap.dedent("""
    第七节 走私、贩卖、运输、制造毒品罪

    第八节 组织、强迫、引诱、容留、介绍卖淫罪

    第九节 制作、贩卖、传播淫秽物品罪

    第七章 危害国防利益罪

    第八章 贪污贿赂罪

    第九章 渎职罪

    第十章 军人违反职责罪

    附则

    # 第一编 总则

    # 第一章 刑法的任务、基本原则和适用范围

    第一条 为了惩罚犯罪，保护人民，根据宪法，结合我国同犯罪作斗争的具体经验及实际情况，制定本法。

    第二条 中华人民共和国刑法的任务，是用刑罚同一切犯罪行为作斗争，以保卫国家安全，保卫人民民主专政的政权和社会主义制度，保护国有财产和劳动群众集体所有的财产，保护公民私人所有的财产，保护公民的人身权利、民主权利和其他权利，维护社会秩序、经济秩序，保障社会主义建设事业的顺利进行。
    """).strip()

# 期望的提取结果
example_extractions = [
    lx.data.Extraction(
        extraction_class="实体",
        extraction_text="走私、贩卖、运输、制造毒品罪",
        attributes={"类型": "罪名", "章节": "第七节", "所属编章": "分则"}
    ),
    lx.data.Extraction(
        extraction_class="实体",
        extraction_text="组织、强迫、引诱、容留、介绍卖淫罪",
        attributes={"类型": "罪名", "章节": "第八节", "所属编章": "分则"}
    ),
    lx.data.Extraction(
        extraction_class="实体",
        extraction_text="制作、贩卖、传播淫秽物品罪",
        attributes={"类型": "罪名", "章节": "第九节", "所属编章": "分则"}
    ),
    lx.data.Extraction(
        extraction_class="实体",
        extraction_text="危害国防利益罪",
        attributes={"类型": "章节标题", "章号": "第七章", "所属编章": "分则"}
    ),
    lx.data.Extraction(
        extraction_class="实体",
        extraction_text="贪污贿赂罪",
        attributes={"类型": "章节标题", "章号": "第八章", "所属编章": "分则"}
    ),
    lx.data.Extraction(
        extraction_class="实体",
        extraction_text="渎职罪",
        attributes={"类型": "章节标题", "章号": "第九章", "所属编章": "分则"}
    ),
    lx.data.Extraction(
        extraction_class="实体",
        extraction_text="军人违反职责罪",
        attributes={"类型": "章节标题", "章号": "第十章", "所属编章": "分则"}
    ),
    lx.data.Extraction(
        extraction_class="实体",
        extraction_text="第一编 总则",
        attributes={"类型": "编标题", "编号": "第一编"}
    ),
    lx.data.Extraction(
        extraction_class="关系描述",
        extraction_text="为了惩罚犯罪，保护人民，根据宪法，结合我国同犯罪作斗争的具体经验及实际情况，制定本法。",
        attributes={
            "类型": "立法目的",
            "法律名称": "刑法",
            "目的": "惩罚犯罪、保护人民",
            "依据": "宪法、实践经验、实际情况"
        }
    ),
    lx.data.Extraction(
        extraction_class="数据指标",
        extraction_text="一切危害国家主权、领土完整和安全，分裂国家、颠覆人民民主专政的政权和推翻社会主义制度，破坏社会秩序和经济秩序，侵犯国有资产或者劳动群众集体所有的财产，侵犯公民私人所有的财产，侵犯公民的人身权利、民主权利和其他权利，以及其他危害社会的行为，依照法律应当受刑罚处罚的，都是犯罪",
        attributes={
            "类型": "犯罪定义",
            "法律条款": "第十三条",
            "涵盖范围": "多类危害行为"
        }
    ),
    lx.data.Extraction(
        extraction_class="数据指标",
        extraction_text="已满十六周岁的人犯罪，应当负刑事责任。已满十四周岁不满十六周岁的人，犯故意杀人、故意伤害致人重伤或者死亡、强奸、抢劫、贩卖毒品、放火、爆炸、投放危险物质罪的，应当负刑事责任。",
        attributes={
            "类型": "刑事责任年龄",
            "法律条款": "第十七条",
            "年龄分组": "3个年龄组",
            "特殊年龄段犯罪": "8种特定犯罪"
        }
    )
]

# 组装为 ExampleData
examples = [
    lx.data.ExampleData(
        text=example_text,
        extractions=example_extractions
    )
]

print("Few-shot 示例定义完成")
print(f"\n示例文本长度: {len(example_text)} 字符")
print(f"示例提取数量: {len(example_extractions)} 个")

# 展示示例提取
print("\n示例提取预览:")
for i, ext in enumerate(example_extractions[:3], 1):
    print(f"{i}. [{ext.extraction_class}] {ext.extraction_text[:50]}...")
    print(f"   属性: {ext.attributes}")

Few-shot 示例定义完成

示例文本长度: 332 字符
示例提取数量: 11 个

示例提取预览:
1. [实体] 走私、贩卖、运输、制造毒品罪...
   属性: {'类型': '罪名', '章节': '第七节', '所属编章': '分则'}
2. [实体] 组织、强迫、引诱、容留、介绍卖淫罪...
   属性: {'类型': '罪名', '章节': '第八节', '所属编章': '分则'}
3. [实体] 制作、贩卖、传播淫秽物品罪...
   属性: {'类型': '罪名', '章节': '第九节', '所属编章': '分则'}


创建 LangExtract 模型并执行提取

In [89]:
# 创建 LangExtract 模型
langextract_model = OpenAILanguageModel(
    model_id=DEFAULT_MODEL,
    api_key=DASHSCOPE_API_KEY,
    base_url=DASHSCOPE_BASE_URL,
)

print("LangExtract 模型创建成功")
print(f"  模型: {DEFAULT_MODEL}")
print(f"  API: {DASHSCOPE_BASE_URL}")

# 准备提取参数
doc_id = "doc_001"
doc_title = PDF_PATH

# 如果 Markdown 文本过长，截断到 8000 字符（避免超过 LLM 上下文限制）
max_length = 8000
if markdown_text and len(markdown_text) > max_length:
    print(f"\n Markdown 文本较长 ({len(markdown_text):,} 字符)，截断至 {max_length:,} 字符")
    extraction_text = markdown_text[:max_length]
else:
    extraction_text = markdown_text

print(f"\n准备提取:")
print(f"  文档 ID: {doc_id}")
print(f"  文档标题: {doc_title}")
print(f"  文本长度: {len(extraction_text):,} 字符" if extraction_text else "  文本长度: 0 字符")

LangExtract 模型创建成功
  模型: qwen3-max
  API: https://dashscope.aliyuncs.com/compatible-mode/v1

准备提取:
  文档 ID: doc_001
  文档标题: /home/crq/llm_scripts/rag/data/刑法4页.pdf
  文本长度: 1,812 字符


执行 LangExtract 提取

In [99]:
# 执行 LangExtract 提取
print("\n正在执行 LangExtract 提取...\n")
extract_start_time = time.time()

if extraction_text:
    result = lx.extract(
        text_or_documents=extraction_text,
        prompt_description=extraction_prompt,
        examples=examples,
        model=langextract_model,
        fence_output=True,  # 要求 LLM 输出用代码块包裹，避免格式错误
        use_schema_constraints=False,  # 不使用严格的 schema 约束，提高灵活性
        prompt_validation_level=PromptValidationLevel.OFF,  # 关闭提示词验证
        show_progress=True,  # 显示提取进度
    )
    
    extract_time = time.time() - extract_start_time
    print(f"\n提取完成 (耗时 {extract_time:.2f}秒)")
else:
    print("没有可提取的文本")
    result = None


正在执行 LangExtract 提取...



LangExtract: model=qwen3-max, current=1,810 chars, processed=0 chars:  [00:53]


提取完成 (耗时 53.99秒)


LangExtract 返回的原始结果需要转换为我们之前定义的 KnowledgeExtraction 数据结构，特别是要提取出 char_interval 字段，这是溯源的关键。

In [100]:
# 转换为 KnowledgeExtraction 对象列表
extractions = []

if result:
    for ext in result.extractions:
        # 提取 char_interval（溯源的核心）
        if ext.char_interval:
            char_interval = {
                "start_pos": ext.char_interval.start_pos,
                "end_pos": ext.char_interval.end_pos
            }
        else:
            char_interval = None
        
        # 创建 KnowledgeExtraction 对象
        ke = KnowledgeExtraction(
            doc_id=doc_id,
            doc_title=doc_title,
            extraction_class=ext.extraction_class,
            extraction_text=ext.extraction_text,
            char_interval=char_interval,
            attributes=ext.attributes or {}
        )
        extractions.append(ke)
    
    print(f"已转换为 KnowledgeExtraction 对象")
    print(f"  提取实体总数: {len(extractions)}")
else:
    print("没有提取结果")

已转换为 KnowledgeExtraction 对象
  提取实体总数: 66


统计实体类型分布

In [101]:
# 统计实体类型分布
if extractions:
    type_counts = Counter(e.extraction_class for e in extractions)
    
    print("\n提取结果统计:")
    print("\n实体类型分布:")
    for entity_type, count in type_counts.most_common():
        percentage = count / len(extractions) * 100
        print(f"   - {entity_type}: {count} 个 ({percentage:.1f}%)")
    
    # 统计溯源情况
    aligned_count = sum(1 for e in extractions if e.char_interval)
    alignment_rate = aligned_count / len(extractions) * 100 if extractions else 0
    
    print(f"\n溯源统计:")
    print(f"   - 有原文位置: {aligned_count} 个")
    print(f"   - 无原文位置: {len(extractions) - aligned_count} 个")
    print(f"   - 对齐率: {alignment_rate:.1f}%")
    
    if alignment_rate < 80:
        print("   对齐率较低，可能影响溯源质量")
    else:
        print("   对齐率良好，溯源质量较高")
    
    # 统计属性情况
    with_attrs = sum(1 for e in extractions if e.attributes)
    attrs_rate = with_attrs / len(extractions) * 100 if extractions else 0
    
    print(f"\n属性统计:")
    print(f"   - 有属性: {with_attrs} 个")
    print(f"   - 无属性: {len(extractions) - with_attrs} 个")
    print(f"   - 属性率: {attrs_rate:.1f}%")
    
    if attrs_rate < 60:
        print("   属性较少，建议优化 Few-shot 示例")
    else:
        print("   属性丰富，有利于知识图谱构建")
else:
    print("没有提取结果可统计")


提取结果统计:

实体类型分布:
   - extraction_text: 23 个 (34.8%)
   - 关系描述: 20 个 (30.3%)
   - 实体: 16 个 (24.2%)
   - 数据指标: 5 个 (7.6%)
   - 事件: 2 个 (3.0%)

溯源统计:
   - 有原文位置: 62 个
   - 无原文位置: 4 个
   - 对齐率: 93.9%
   对齐率良好，溯源质量较高

属性统计:
   - 有属性: 42 个
   - 无属性: 24 个
   - 属性率: 63.6%
   属性丰富，有利于知识图谱构建


展示前5个提取示例（含溯源验证）

In [102]:
import json

# 展示前5个提取示例（含溯源验证）
if extractions and extraction_text:
    print("\n提取示例（前5个，含溯源验证）:")
    print("=" * 80)
    
    for i, ext in enumerate(extractions[:5], 1):
        print(f"\n[{i}] {ext.extraction_class}: {ext.extraction_text[:50]}{'...' if len(ext.extraction_text) > 50 else ''}")
        
        if ext.attributes:
            print(f"    属性: {ext.attributes}")
        
        if ext.char_interval:
            interval = ext.char_interval
            print(f"    原文位置: 字符 {interval['start_pos']} - {interval['end_pos']}")
            
            # 验证：从 Markdown 中提取对应位置的文本
            original_text = extraction_text[interval['start_pos']:interval['end_pos']]
            if original_text == ext.extraction_text:
                print(f"    溯源验证通过")
            else:
                # 修正了切片显示逻辑，避免原文太长时显示混乱
                print(f"    溯源不匹配: 原文为 '{original_text[:30]}...'")
        else:
            print(f"    无原文位置信息")
    
    print("\n" + "=" * 80)
    
    # 保存提取结果
    extractions_file = OUTPUT_DIR / "chapter5_extractions.json"
    with open(extractions_file, "w", encoding="utf-8") as f:
        # 假设 e 有 to_dict 方法，如果没有可以改用 vars(e)
        json.dump([e.to_dict() for e in extractions], f, ensure_ascii=False, indent=2)
    
    print(f"\n提取结果已保存: {extractions_file}")
    print(f"   文件大小: {extractions_file.stat().st_size / 1024:.2f} KB")
else:
    print("没有提取结果可展示")


提取示例（前5个，含溯源验证）:

[1] 实体: 走私、贩卖、运输、制造毒品罪
    属性: {'类型': '罪名', '章节': '第七节', '所属编章': '分则'}
    原文位置: 字符 4 - 18
    溯源验证通过

[2] extraction_text: 走私、贩卖、运输、制造毒品罪
    原文位置: 字符 4 - 18
    溯源验证通过

[3] 实体: 组织、强迫、引诱、容留、介绍卖淫罪
    属性: {'类型': '罪名', '章节': '第八节', '所属编章': '分则'}
    原文位置: 字符 24 - 41
    溯源验证通过

[4] extraction_text: 组织、强迫、引诱、容留、介绍卖淫罪
    原文位置: 字符 24 - 41
    溯源验证通过

[5] 实体: 制作、贩卖、传播淫秽物品罪
    属性: {'类型': '罪名', '章节': '第九节', '所属编章': '分则'}
    原文位置: 字符 47 - 60
    溯源验证通过


提取结果已保存: outputs/chapter5_extractions.json
   文件大小: 30.00 KB


### ChromaDB 向量存储

步骤1：初始化 ChromaDB 和 Embeddings

In [104]:
from pathlib import Path
import chromadb
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# 初始化 Embeddings
embeddings = OpenAIEmbeddings(
    model=EMBEDDING_MODEL,
    api_key=DASHSCOPE_API_KEY,
    base_url=DASHSCOPE_BASE_URL,
    check_embedding_ctx_length=False,  # 禁用 token 长度检查
    chunk_size=10,  # 阿里云百炼限制批量大小不超过 10
)
print(f"Embeddings: {EMBEDDING_MODEL}")

# 初始化 ChromaDB
persist_dir = Path(CHROMA_PERSIST_DIR)
persist_dir.mkdir(parents=True, exist_ok=True)
client = chromadb.PersistentClient(path=str(persist_dir))
print(f"ChromaDB: {persist_dir}")

# 重建集合（删除旧数据）
try:
    client.delete_collection(name=COLLECTION_NAME)
    print(f"  已删除旧集合")
except Exception:
    # 如果集合不存在，删除会报错，这里直接跳过
    pass

# 创建 LangChain 的 Chroma 向量存储
vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    embedding_function=embeddings,
    client=client,
)
print(f"  已创建集合: {COLLECTION_NAME}\n")

Embeddings: text-embedding-v3
ChromaDB: chroma_db_chapter5
  已创建集合: pdf_knowledge_graph



/tmp/ipykernel_2599212/788226479.py:31: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


步骤 2：将提取结果存入向量库（含溯源信息）

 这是关键步骤：我们需要将每个提取结果转换为向量，并在 metadata 中保存完整的溯源信息（char_interval）。

In [105]:
import uuid
import json

# 准备向量化数据
if extractions:
    print(f"存储到 ChromaDB...")
    
    # 假设 ext 对象拥有 to_searchable_text 方法
    texts = [ext.to_searchable_text() for ext in extractions]
    metadatas = []
    ids = []
    
    for ext in extractions:
        # 生成唯一 ID
        ids.append(str(uuid.uuid4()))
        
        # 关键：在 metadata 中保存溯源信息
        metadatas.append({
            "doc_id": ext.doc_id,
            "doc_title": ext.doc_title,
            "extraction_class": ext.extraction_class,
            "extraction_text": ext.extraction_text,
            # 将字典转为 JSON 字符串，因为 ChromaDB 的 metadata 仅支持简单类型（str, int, float, bool）
            "char_interval": json.dumps(ext.char_interval) if ext.char_interval else "", 
            "attributes": json.dumps(ext.attributes, ensure_ascii=False),
        })
    
    # 批量插入向量
    vectorstore.add_texts(texts=texts, metadatas=metadatas, ids=ids)
    print(f"✓ 已索引 {len(texts)} 条记录\n")
    
    # 查询集合信息
    collection = client.get_collection(COLLECTION_NAME)
    count = collection.count()
    print(f"集合统计:")
    print(f"   - 向量数量: {count}")
    print(f"   - 集合名称: {COLLECTION_NAME}")
else:
    print("没有提取结果可存储")

存储到 ChromaDB...
✓ 已索引 66 条记录

集合统计:
   - 向量数量: 66
   - 集合名称: pdf_knowledge_graph


### 知识图谱构建

In [106]:
import json

# 构建知识图谱
knowledge_graph = {
    "entities": {},
    "relations": []
}

if extractions:
    for ext in extractions:
        if ext.extraction_class == "关系描述":
            # 提取关系
            attrs = ext.attributes or {} # 防止 attributes 为 None
            knowledge_graph["relations"].append({
                "text": ext.extraction_text,
                "type": attrs.get("类型", "未知"),
                "subject": attrs.get("主体1"),
                "object": attrs.get("主体2"),
                "relation": attrs.get("关系"),
                "source": ext.doc_title
            })
        elif ext.extraction_class in ["实体", "数据指标"]:
            # 提取实体（保留溯源信息）
            entity_name = ext.extraction_text
            if entity_name not in knowledge_graph["entities"]:
                knowledge_graph["entities"][entity_name] = {
                    "type": ext.extraction_class,
                    "attributes": ext.attributes,
                    "mentions": []  # 存储所有提及位置（溯源）
                }
            # 添加提及位置
            knowledge_graph["entities"][entity_name]["mentions"].append({
                "source": ext.doc_title,
                "position": ext.char_interval  # 溯源关键
            })
    
    print(f"知识图谱统计:")
    print(f"   - 实体数: {len(knowledge_graph['entities'])}")
    print(f"   - 关系数: {len(knowledge_graph['relations'])}")
    
    # 保存知识图谱
    kg_file = OUTPUT_DIR / "knowledge_graph.json"
    with open(kg_file, "w", encoding="utf-8") as f:
        # 补全了保存逻辑
        json.dump(knowledge_graph, f, ensure_ascii=False, indent=2)
    
    print(f"✓ 知识图谱已保存至: {kg_file}")
else:
    print("没有提取结果可构建知识图谱")

知识图谱统计:
   - 实体数: 21
   - 关系数: 20
✓ 知识图谱已保存至: outputs/knowledge_graph.json


### Agent 智能问答与溯源

步骤 1：定义图检索函数（含溯源信息）

In [107]:
import json

# 定义向量检索函数（含溯源信息）
def search_knowledge(query: str, top_k: int = 5):
    """从向量库检索相关知识，返回含溯源信息的结果"""
    # 假设 extractions 是全局变量或已在外部定义
    # 如果只是为了检查库是否初始化，通常建议检查 vectorstore 是否为空
    if 'vectorstore' not in globals() or vectorstore is None:
        print("警告: 向量库未初始化")
        return []
    
    # 执行相似度搜索 (Chroma 默认返回 L2 距离，score 越小越相似)
    results = vectorstore.similarity_search_with_score(query, k=top_k)
    
    # 格式化结果（包含溯源信息）
    formatted_results = []
    for doc, score in results:
        # L2 距离转换为相似度分数（仅作参考）
        similarity_score = 1 / (1 + score) 
        
        # 解析溯源信息
        char_interval_str = doc.metadata.get("char_interval", "")
        char_interval = json.loads(char_interval_str) if char_interval_str else None
        
        formatted_results.append({
            "score": similarity_score,
            "doc_title": doc.metadata.get("doc_title"),
            "extraction_class": doc.metadata.get("extraction_class"),
            "extraction_text": doc.metadata.get("extraction_text"),
            "char_interval": char_interval,  # 溯源关键
            "attributes": json.loads(doc.metadata.get("attributes", "{}")),
        })
    
    return formatted_results

print("向量检索函数定义完成")

向量检索函数定义完成


In [108]:
import json

# 定义向量检索函数（含溯源信息）
def search_knowledge(query: str, top_k: int = 5):
    """从向量库检索相关知识，返回含溯源信息的结果"""
    
    # 检查向量库对象是否存在
    if 'vectorstore' not in globals() or vectorstore is None:
        print("警告: 向量库未初始化")
        return []
    
    # 执行相似度搜索 (Chroma 默认返回 L2 距离，score 越小代表向量间距越近)
    # 使用 similarity_search_with_score 可以同时获取文档内容和距离分数
    results = vectorstore.similarity_search_with_score(query, k=top_k)
    
    # 格式化结果（包含溯源信息）
    formatted_results = []
    for doc, score in results:
        # 将 L2 距离转换为直观的相似度分数（仅作参考，距离越小分数越接近 1）
        similarity_score = 1 / (1 + score) 
        
        # 从 metadata 中安全获取并解析溯源信息
        # 注意：ChromaDB 存储时 metadata 的值通常为字符串，需要 json.loads 转回字典
        char_interval_raw = doc.metadata.get("char_interval", "")
        char_interval = json.loads(char_interval_raw) if char_interval_raw else None
        
        attributes_raw = doc.metadata.get("attributes", "{}")
        attributes = json.loads(attributes_raw) if isinstance(attributes_raw, str) else attributes_raw
        
        formatted_results.append({
            "score": similarity_score,
            "doc_title": doc.metadata.get("doc_title"),
            "extraction_class": doc.metadata.get("extraction_class"),
            "extraction_text": doc.metadata.get("extraction_text"),
            "char_interval": char_interval,  # 溯源关键偏移量
            "attributes": attributes,        # 实体属性详情
        })
    
    return formatted_results

print("向量检索函数定义完成")

向量检索函数定义完成


定义graph_search函数

In [116]:
# =============================================================================
# 图检索函数（GraphRAG 核心）
# =============================================================================

def graph_search(query_entity: str, hop: int = 1):
    """
    从知识图谱中检索实体及其关联关系
    
    Args:
        query_entity: 查询的实体名称（支持模糊匹配）
        hop: 跳数，1表示直接关联，2表示二度关联
    
    Returns:
        相关实体和关系
    """
    results = {
        "matched_entities": [],
        "related_relations": [],
        "connected_entities": set()
    }
    
    # 1. 模糊匹配实体
    for entity_name, entity_data in knowledge_graph["entities"].items():
        if query_entity.lower() in entity_name.lower():
            results["matched_entities"].append({
                "name": entity_name,
                **entity_data
            })
    
    # 2. 查找相关关系
    matched_names = [e["name"] for e in results["matched_entities"]]
    

print("图检索函数 definition 定义完成")

图检索函数 definition 定义完成


定义工具

In [117]:
# =============================================================================
# LangChain 1.1 GraphRAG Tools & Agent 完整版
# =============================================================================

from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.messages import HumanMessage

# -----------------------------------------------------------------------------
# 1. 定义检索工具 (Tools)
# -----------------------------------------------------------------------------

@tool
def vector_search_tool(query: str) -> str:
    """
    向量语义检索：根据问题搜索相关知识片段。
    返回语义相似的文档内容和溯源信息。
    适合：查找与问题语义相关的内容。
    """
    # 假设 search_knowledge 是底层定义的检索函数
    results = search_knowledge(query, top_k=5)
    
    if not results:
        return "未找到相关信息"
    
    output_parts = []
    for i, r in enumerate(results, 1):
        part = f"[V{i}] {r['extraction_class']}: {r['extraction_text']}"
        if r.get('char_interval'):
            interval = r['char_interval']
            part += f"\n     位置: 字符 {interval['start_pos']}-{interval['end_pos']}"
        part += f"\n     来源: {r['doc_title']}"
        if r.get('attributes'):
            part += f"\n     属性: {r['attributes']}"
        output_parts.append(part)
    
    return "\n\n".join(output_parts)


@tool
def graph_search_tool(entity: str) -> str:
    """
    知识图谱检索：根据实体名称查找相关实体和关系。
    用于发现实体之间的结构化关联。
    适合：查找某个实体相关的关系、关联实体。
    """
    # 假设 graph_search 是底层定义的图检索函数
    results = graph_search(entity, hop=1)
    
    if not results:
        return f"未找到与 '{entity}' 相关的图谱信息"

    output_parts = []
    # 假设 results 是包含关系字典的列表
    for i, res in enumerate(results, 1):
        output_parts.append(f"[G{i}] {res}")
    
    return "\n".join(output_parts)


@tool
def hybrid_search_tool(query: str) -> str:
    """
    混合检索（GraphRAG）：同时进行向量语义检索和知识图谱检索。
    适合：复杂问题，需要结合语义相似和结构化关系。
    """
    # 1. 向量检索
    vector_result = vector_search_tool.invoke(query)
    
    # 2. 从问题中提取可能的实体进行图检索（简单分词处理）
    words = [w for w in query.replace("？", "").replace("?", "").replace("，", " ").replace("。", " ").split() if len(w) >= 2]
    
    graph_results = []
    for word in words[:3]:
        gr = graph_search_tool.invoke(word)
        if "未找到" not in gr:
            graph_results.append(gr)
    
    # 3. 组合结果
    output = "=== 向量检索结果 ===\n" + vector_result

步骤 2：定义 Agent 问答函数（含溯源上下文）

In [118]:
# =============================================================================
# 创建 LangChain 1.1 GraphRAG Agent
# =============================================================================

from langchain_openai import ChatOpenAI

# 创建 DeepSeek 模型实例
llm = ChatOpenAI(
    model=DEFAULT_MODEL,
    api_key=DASHSCOPE_API_KEY,
    base_url=DASHSCOPE_BASE_URL,
    temperature=0.3
)

# 创建 Agent
graphrag_agent = create_agent(
    model=llm,
    tools=[vector_search_tool, graph_search_tool, hybrid_search_tool],
    system_prompt="""你是一个 GraphRAG 知识图谱问答助手。

你有以下工具可用：
1. vector_search_tool - 向量语义检索，找语义相似的内容
2. graph_search_tool - 图谱检索，根据实体名找关系
3. hybrid_search_tool - 混合检索，同时使用向量和图谱

回答策略：
- 简单的内容查询：用 vector_search_tool
- 查找实体关系：用 graph_search_tool  
- 复杂问题：用 hybrid_search_tool

回答要求：
1. 综合检索到的信息回答问题
2. 标注信息来源（如 [V1] 表示向量结果，[G1] 表示图谱关系）
3. 如果有溯源位置信息，也一并说明
"""
)

print("✓ GraphRAG Agent 创建完成")

✓ GraphRAG Agent 创建完成


In [119]:
# =============================================================================
# 封装 agent_query 函数
# =============================================================================

def agent_query(question: str, top_k: int = 5):
    """
    GraphRAG Agent 问答
    
    Args:
        question: 用户问题
        top_k: 检索数量
    
    Returns:
        包含 question, answer, evidence 的字典
    """
    # 调用 Agent
    result = graphrag_agent.invoke({
        "messages": [HumanMessage(content=question)]
    })
    
    # 提取最终回答
    answer = result["messages"][-1].content
    
    # 提取工具调用记录作为 evidence
    tool_calls = []
    for msg in result["messages"]:
        # 工具调用请求
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            for tc in msg.tool_calls:
                tool_calls.append({
                    "type": "call",
                    "tool": tc.get("name", "unknown"),
                    "args": tc.get("args", {})
                })
        # 工具返回结果
        if hasattr(msg, "name") and msg.name:
            tool_calls.append({
                "type": "result",
                "tool": msg.name,
                "content": msg.content[:500] + "..." if len(msg.content) > 500 else msg.content
            })
            
    return {
        "question": question,
        "answer": answer,
        "evidence": tool_calls
    }

print("✓ agent_query 函数定义完成")

✓ agent_query 函数定义完成


步骤 3：测试 Agent 问答（展示完整溯源）

In [120]:
# =============================================================================
# 测试 GraphRAG 问答
# =============================================================================

test_questions = [
    "刑法的原则是什么？",
    "外国人的刑事责任怎么追究？",
    "未成年人犯罪完全不追究吗?",
    "刑法第十三条是什么?"
]

for question in test_questions:
    print(f"\n{'='*80}")
    print(f"问题: {question}")
    print(f"{'-'*80}")
    
    result = agent_query(question, top_k=3)
    
    print(f"\n回答:\n{result['answer']}\n")
    
    # 展示工具调用证据
    if result['evidence']:
        print(f"Agent 工具调用记录:")
        for i, ev in enumerate(result['evidence'], 1):
            if ev['type'] == 'call':
                print(f"   [{i}] 调用 {ev['tool']}")
                print(f"       参数: {ev['args']}")
            else:
                print(f"   [{i}] {ev['tool']} 返回:")
                # 只显示前200字符
                content = ev['content']
                if len(content) > 200:
                    content = content[:200] + "..."
                print(f"       {content}")
    
    print(f"\n{'='*80}")


问题: 刑法的原则是什么？
--------------------------------------------------------------------------------

回答:
根据《中华人民共和国刑法》第一章的内容，刑法的基本原则体现在其任务和立法目的中。具体包括以下几点：

1. **惩罚犯罪，保护人民**：刑法的制定目的是为了惩罚犯罪行为，同时保护人民的合法权益 [V5]。
2. **保卫国家安全与政权**：通过刑罚手段同一切犯罪行为作斗争，以保卫国家安全、人民民主专政的政权和社会主义制度 [V3/V4]。
3. **保护各类财产和权利**：包括国有财产、集体财产、公民私人财产，以及公民的人身权利、民主权利和其他权利 [V3/V4]。
4. **维护社会和经济秩序**：确保社会秩序和经济秩序稳定，保障社会主义建设事业顺利进行 [V3/V4]。

这些内容共同构成了刑法的基本原则框架，体现了刑法在国家治理和社会保护中的核心作用。

Agent 工具调用记录:
   [1] 调用 vector_search_tool
       参数: {'query': '刑法的原则'}
   [2] vector_search_tool 返回:
       [V1] extraction_text: 第一章 刑法的任务、基本原则和适用范围
     位置: 字符 124-143
     来源: /home/crq/llm_scripts/rag/data/刑法4页.pdf

[V2] 实体: 第一章 刑法的任务、基本原则和适用范围
     位置: 字符 124-143
     来源: /home/crq/llm_scripts/rag/data...


问题: 外国人的刑事责任怎么追究？
--------------------------------------------------------------------------------

回答:
根据中国法律，对外国人的刑事责任追究主要遵循以下原则和程序：

1. **属地管辖原则**：凡在中国领域内犯罪的外国人，除享有外交特权和豁免权的人员外，均适用中国刑法。这是《中华人民共和国刑法》第6条明确规定的 [V1]。

2. **外交豁免例外**：对于享有外交特权和豁免权的外国人（